<a href="https://colab.research.google.com/github/rickqiu/trax_chatbots/blob/main/reformer_chatbots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License")
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

 https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Chatbot with Reformer Model
@author: Rick Qiu

This notebook demonstrates a chatbot with a Reformer model pre-trained on the MultiWoz dataset. The chatbot is built with Google Trax, which is a low-code and high-speed deep learning library.

Although the chatbot can not compete with Alexa, Siri, Cortana and Meena, it gives reasonable good conversations in the domains of attraction, hotel, taxi, train, hospital and police.

Through this notebook, not only one will learn how to create an AI chatbot with few lines of code, but one will also have the opportunities of seeing the algorithm i.e., the model structure and chatting to the AI.

### References

- [MultiWoz](https://arxiv.org/abs/1810.00278) dataset
- [Reformer](https://arxiv.org/abs/2001.04451) paper
- [Trax](https://github.com/google/trax) code repository









## 1. Setup

In [1]:
!pip install -q -U https://storage.googleapis.com/jax-releases/cuda101/jaxlib-0.1.55-cp36-none-manylinux2010_x86_64.whl
!pip install -q -U jax
!pip install -q -U trax

     |████████████████████████████████| 144.8MB 27kB/s 
     |████████████████████████████████| 491kB 6.5MB/s 
     |████████████████████████████████| 419kB 4.7MB/s 
     |████████████████████████████████| 2.6MB 15.4MB/s 
     |████████████████████████████████| 174kB 44.3MB/s 
     |████████████████████████████████| 1.5MB 46.7MB/s 
     |████████████████████████████████| 1.3MB 48.4MB/s 
     |████████████████████████████████| 1.1MB 52.3MB/s 
     |████████████████████████████████| 3.6MB 58.1MB/s 
     |████████████████████████████████| 348kB 68.3MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
     |████████████████████████████████| 655kB 63.9MB/s 
     |████████████████████████████████| 368kB 60.6MB/s 
     |████████████████████████████████| 983kB 53.7MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
     |████████████████████████████████| 358kB 61.0MB/s 
     |████████████████████████████████| 5.3MB 60.1MB/s 
     |████████████████████████████████| 194kB 62.1MB

In [2]:
!pip list | grep trax

trax                          1.3.5                


In [3]:
# clone to get vocabs and the pretrained model parts from repository
!git clone https://github.com/rickqiu/trax_chatbots.git

Cloning into 'trax_chatbots'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 89 (delta 43), reused 0 (delta 0), pack-reused 17
Unpacking objects: 100% (89/89), done.
Checking out files: 100% (11/11), done.


In [4]:
%cd trax_chatbots
!tar -xzvf vocabs.tar.gz
!cat model_splits/* > chatbot_model1.pkl.gz
%cd ..

/content/trax_chatbots
vocabs/
vocabs/en_32k.sentencepiece
vocabs/en_32k.sentencepiece.vocab
vocabs/en_32k.subword
vocabs/en_8k.subword
/content


In [5]:
# import libraries
import numpy as np
import trax   
from trax import layers as tl

## 2. Modelling

In [27]:
# define attention for fast inference
def attention(*args, **kwargs):
    kwargs['predict_mem_len'] = 240
    kwargs['predict_drop_len'] = 120
    return tl.SelfAttention(*args, **kwargs)

# define the model
model = trax.models.reformer.ReformerLM( 
        vocab_size=33000,
        n_layers=6,
        mode='predict', # default 'train' for model training
        attention_type=attention
    )

In [28]:
# display the Reformer model
print(str(model))

Serial[
  ShiftRight(1)
  Embedding_33000_512
  Dropout
  PositionalEncoding
  Dup_out2
  ReversibleSerial_in2_out2[
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        FastGelu
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        FastGelu
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        

## 3. Using pre-traind model

In [29]:
# define an input signature
shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)

# initialize the model from file
model.init_from_file('trax_chatbots/chatbot_model1.pkl.gz',
                     weights_only=True, input_signature=shape11)

# save the starting state
STARTING_STATE = model.state

## 4. Code for making prediction

In [30]:
# https://www.deeplearning.ai/natural-language-processing-specialization/
# vocabulary file directory
VOCAB_DIR = './trax_chatbots/vocabs'

# vocabulary filename
VOCAB_FILE = 'en_32k.subword'

def tokenize(sentence, vocab_file, vocab_dir):
    return list(trax.data.tokenize(iter([sentence]), vocab_file=vocab_file, vocab_dir=vocab_dir))[0]


def detokenize(tokens, vocab_file, vocab_dir):
    return trax.data.detokenize(tokens, vocab_file=vocab_file, vocab_dir=vocab_dir)


def generate_output(model, start_sentence, vocab_file, vocab_dir, temperature):
    """
    Args:
        model:  the Reformer language model you just trained
        start_sentence (string): starting sentence of the conversation
        vocab_file (string): vocabulary filename
        vocab_dir (string): directory of the vocabulary file
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)

    Returns:
        generator: yields the next symbol generated by the model
    """
    
    input_tokens =  tokenize(start_sentence, vocab_file, vocab_dir)
    
    # add batch dimension to array
    input_tokens_with_batch = np.expand_dims(input_tokens, axis=0)
    
    # call the autoregressive_sample_stream function from trax
    output_gen = trax.supervised.decoding.autoregressive_sample_stream( 
        model,
        inputs=input_tokens_with_batch,
        temperature=temperature
    )
    
    return output_gen


def generate_sentence(model, model_state, start_sentence, vocab_file, vocab_dir, temperature):
    """
    Args:
        model:  the Reformer language model you just trained
        model_state (np.array): initial state of the model before decoding
        start_sentence (string): starting sentence of the conversation
        vocab_file (string): vocabulary filename
        vocab_dir (string): directory of the vocabulary file
        temperature (float): parameter for sampling ranging from 0.0 to 1.0.
            0.0: same as argmax, always pick the most probable token
            1.0: sampling from the distribution (can sometimes say random things)

    Returns:
        generator: yields the next symbol generated by the model
    """  
    
    # define the delimiters we used during training
    delimiter_1 = 'Person 1: ' 
    delimiter_2 = 'Person 2: '
    
    # initialize detokenized output
    sentence = ''

    # output tokens
    result = []
    
    # reset the model state when starting a new dialogue
    model.state = model_state
    
    # calls the output generator implemented earlier
    output = generate_output(model, start_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, temperature=temperature)
    
    # print the starting sentence
    #print(start_sentence.split(delimiter_2)[0].strip())
    
    # loop below yields the next tokens until max_len is reached. the if-elif is just for prettifying the output.
    for o in output:
        
        result.append(o)
        
        sentence = detokenize(np.concatenate(result, axis=0), vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)
        
        if sentence.endswith(delimiter_1):
            sentence = sentence.split(delimiter_1)[0]
            print(f'{delimiter_2}{sentence}')
            break
        elif sentence.endswith(delimiter_2):
            sentence = sentence.split(delimiter_2)[0]
            print(f'{delimiter_1}{sentence}')
            break

## 5. Making a conversatoin with AI

Run the following cell to start a conversation input. Enter 'q' to exit.

In [36]:
sample_sentence = ''
in_msg = ''

while True:
  in_msg = input('Person 1: ')
  if in_msg == 'q':
    break
    
  sample_sentence += ' Person 1: ' + in_msg + ' Person 2: '
  generate_sentence(model, model_state=STARTING_STATE, 
  start_sentence=sample_sentence,
  vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, 
  temperature=0.2)

Person 1: Are there theatres in town?
Person 2: There are 4 theatres in town. Do you have a preference on area? 
Person 1: No, I don't care. Which one would you recommend?
Person 2: I would recommend the Mumford Theatre. It's located at Anglia Ruskin Enterprise, east road. 
Person 1: Yes, could I get the postcode and phone number?
Person 2: The phone number is 01223332360 and the postcode is cb58as. 
Person 1: Thank you very much.
Person 2: You're welcome. The postcode is cb58as. 
Person 1: q
